In [24]:
import os
import re
import pandas as pd

In [25]:
# change working directory to the /Users/tildeidunsloth/Desktop/DatSci_25/DatSci_25/cleaned
os.chdir('/Users/tildeidunsloth/Desktop/DatSci_25/DatSci25/data/cleaned/')

In [26]:
# load in csv file with pandas
df = pd.read_csv('parliament_speeches.csv')

In [27]:
# show ten first rows
print(df.head(10))

# check length of dataframe
print(len(df))

# check how many rows for each unique year
print(df['year'].value_counts())

# show rows with year 2000
print(df[df['year'] == 2000])

   year  meeting_number                speaker  \
0  2013              45         Valgs Prøvelse   
1  2013              45         Valgs Prøvelse   
2  2013              45              Formanden   
3  2013              45              Formanden   
4  2013              45         Valgs Prøvelse   
5  2013              45  Forhandling Formanden   
6  2013              45   Afstemning Formanden   
7  2013              45         Valgs Prøvelse   
8  2013              45  Forhandling Formanden   
9  2013              45   Afstemning Formanden   

                                              speech  
0  Godkendelse af stedfortræder som midlertidigt ...  
1  Godkendelse af stedfortræder som repræsentant ...  
2  Mødet er åbnet. I dag er der følgende anmeldel...  
3  Anne Baastrup (SF) har søgt om orlov på grund ...  
4  Godkendelse af stedfortræder som midlertidigt ...  
5  Fra Udvalget til Valgs Prøvelse har jeg modtag...  
6  Jeg slutter afstemningen. Udvalgets indstillin...  
7  Godken

In [28]:
# make a subset of rows where the word "klima" appears in the speech
klima_df = df[df['speech'].str.contains('klima', case=False, na=False)]

In [29]:
print(len(klima_df))
print(klima_df.head(10))

18466
     year  meeting_number                                            speaker  \
229  2013              45                                Christian Juhl (EL)   
450  2013              51                                     (Bent Bøgsted)   
454  2013              51                       Lars Christian Lilleholt (V)   
456  2013              51                                          Theil (S)   
460  2013              51                             Andreas Steenberg (RV)   
462  2013              51                                    Steen Gade (SF)   
464  2013              51                                   Per Clausen (EL)   
467  2013              51                                     (Bent Bøgsted)   
468  2013              51  Klima Energi Og Bygningsministeren (Rasmus Hel...   
469  2013              51                                     (Bent Bøgsted)   

                                                speech  
229  Jeg tror ikke, at de fleste arbejdere i Danmar...  

In [30]:
# save klima_df to a new csv file
klima_df.to_csv('klima_speeches.csv', index=False)